# Protocol of experiments 08.05.2017

__Goal:__ Provide proof of concept and working example for the idea of running ToolDog analyser in the Docker container

__Requirements:__
1. Installed [Docker](https://store.docker.com/search?type=edition&offering=community)
1. Installed Python3, pip

    

First, we import a library for running bash commands from Python code.

In [85]:
import subprocess

Then we define two util functions.

First one if for running bash command.

In [86]:
def run_command(command):
    """Run bash command and return the result

    :param str command: String representation of bash command
    
    :return: Returns a generator of output of the result of running bash command in bytes
    :rtype: iter
    """
    command = command.split()
    p = subprocess.Popen(command,
                         stdout=subprocess.PIPE,
                         stderr=subprocess.STDOUT)
    return iter(p.stdout.readline, b'')

Second one is for writing string in the file.

In [87]:
def write_to_file(filename, string = ''):
    """Write string to file

    :param str filename: Filename
    :param str string: String to write
    """
    f = open(filename, 'w')
    f.write(string)
    f.close()

Than we define some constants for future use.

In [88]:
PYTHON_VERSION = 3 # version of python tool

In [89]:
TOOL_NAME = 'tooldog' # name of the tool, assuming it is available via PyPi

In [90]:
OUTPUT_FORMAT = 'cwl' # output: cwl / xml

In [91]:
OUTPUT_FILENAME = TOOL_NAME + "." + OUTPUT_FORMAT # output filename

In [92]:
PYTHON_PATH = "/usr/local/lib/python3.5/dist-packages/" if PYTHON_VERSION == 3 else \
"/usr/local/lib/python2.7/dist-packages/" # PYTHONPATH, required for argparse2tool

Compose bash command for running detached docker container from [tooldog-analyser-sandbox](https://hub.docker.com/r/inkuzm/tooldog-analyser-sandbox/) image, that will be destroyed after stopping.

In [93]:
cmd = 'docker run -d -e PYTHONPATH=' + PYTHON_PATH + ' --rm inkuzm/tooldog-analyser-sandbox tail -f /dev/null'

Save container ID for future use.

In [94]:
CONTAINER_ID = ""
for line in run_command(cmd):
    CONTAINER_ID = line.decode("utf-8")[:-1]
    print(CONTAINER_ID)

bd47bbd3272a48739f8afadae8d24723a0508e330f41715cba60fe4dad953936


Yay! Now we can do stuff inside the container.

Let's define one more util function for running commands inside the created container.

In [95]:
def exec_command_in_container(command, cid):
    """Run bash command inside container cid

    :param str command: String representation of bash command
    
    :return: Returns a generator of output of the result of running bash command in bytes
    :rtype: iter
    """  
    cmd = 'docker exec ' + cid + ' ' + command
    return run_command(cmd)

We install argparse2tool:

In [96]:
for line in exec_command_in_container('pip3 install argparse2tool', CONTAINER_ID):
    print(line.decode("utf-8")[:-1])

  Running setup.py bdist_wheel for argparse2tool: started
  Running setup.py bdist_wheel for argparse2tool: finished with status 'error'
  Complete output from command /usr/bin/python3 -u -c "import setuptools, tokenize;__file__='/tmp/pip-build-0do37j9w/argparse2tool/setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" bdist_wheel -d /tmp/tmplqq66w3xpip-wheel- --python-tag cp35:
  /usr/lib/python3/dist-packages/setuptools/dist.py:285: UserWarning: Normalizing '0.4.3-2' to '0.4.3.post2'
    normalized_version,
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build/lib
  creating build/lib/argparse
  copying argparse/argparse_cwl_translation.py -> build/lib/argparse
  copying argparse/argparse_galaxy_translation.py -> build/lib/argparse
  copying argparse/__init__.py -> build/lib/argparse
  creating build/lib/argparse2tool
  copying argparse2tool/__init__.py -> build/

  Running setup.py install for argparse2tool: started
    Running setup.py install for argparse2tool: finished with status 'done'


As well as the tool for analysing:

In [97]:
for line in exec_command_in_container('pip3 install ' + TOOL_NAME, CONTAINER_ID):
    print(line.decode("utf-8")[:-1])

  Running setup.py bdist_wheel for tooldog: started
  Running setup.py bdist_wheel for tooldog: finished with status 'done'
  Stored in directory: /root/.cache/pip/wheels/69/61/26/59bf47f3f6cb6f86fe97126d5e9eb5f5edbf8a576be3c8d890
  Running setup.py bdist_wheel for cwlgen: started
  Running setup.py bdist_wheel for cwlgen: finished with status 'done'
  Stored in directory: /root/.cache/pip/wheels/ef/59/eb/de39a2226114e01698e0b63c35a6d11d5b4bc20fc4c37b8c2d
  Running setup.py bdist_wheel for isodate: started
  Running setup.py bdist_wheel for isodate: finished with status 'done'
  Stored in directory: /root/.cache/pip/wheels/61/c0/d2/6b4a10c222ba9261ab9872a8f05d471652962284e8c677e5e7
Successfully built tooldog cwlgen isodate


Now we are ready to generate CWL:

In [98]:
cwl_tool = '' # string representation of cwl tool
for line in exec_command_in_container('tooldog --generate_cwl_tool', CONTAINER_ID):
    cwl_tool += line.decode("utf-8")[:-1]

Write CWL to file:

In [99]:
write_to_file(OUTPUT_FILENAME, cwl_tool)

Finally, let's stop the container. It will be removed automatically.

In [100]:
run_command('docker stop ' + CONTAINER_ID)